# Plotting Energy Density Transform

In [ ]:
from data import DEData
import torch
import numpy as np
import matplotlib.gridspec as gridspec
import matplotlib.pyplot as plt

In [ ]:
dataset = DEData('dE_data_2')

data_loader = torch.utils.data.DataLoader(
    dataset=dataset,
    shuffle=True
)

In [ ]:
model = torch.load('dE_model.pt')
model.eval()

In [ ]:
graph_1_sums = []
graph_2_sums = []

for i, data in enumerate(data_loader):
    dE_initial = data[0]
    dNch_deta_final = data[1]
    x_axis = dataset.data_axis

    outputs = model(dE_initial.float())

    indices = []
    sum_x_axis = []

    for i, eta in enumerate(x_axis):
        if -4.9 < x_axis[i] < -4:
            indices.append(i)
            sum_x_axis.append(eta)

    for i, p in enumerate(dNch_deta_final):
        graph_1_sums.append( ( np.trapz(dE_initial[i][indices[0] : indices[-1] + 1], x=sum_x_axis), np.trapz(p[indices[0] : indices[-1] + 1], x=sum_x_axis) )  )

        graph_2_sums.append( ( np.trapz(outputs.detach().numpy()[i][indices[0] : indices[-1] + 1], x=sum_x_axis), np.trapz(p[indices[0] : indices[-1] + 1], x=sum_x_axis) )  )

In [ ]:
gs = gridspec.GridSpec(1, 2)
fig = plt.figure(figsize=(15, 5))
fig.suptitle("Model vs Actual Output Comparison")

ax0 = fig.add_subplot(gs[0, 0])

for sum in graph_1_sums:
    ax0.plot(sum[0], sum[1], 'x', color='red')
ax0.set_xlabel('Initial Input')
ax0.set_ylabel('Actual Output')

ax1 = fig.add_subplot(gs[0, 1])

for sum in graph_2_sums:
    ax1.plot(sum[0], sum[1], 'x', color='blue')

ax1.set_xlabel('Generated Output')
ax1.set_ylabel('Actual Output')

plt.show()